## Forge images

In [ ]:
from scipy import ndimage
from PIL import Image
from tqdm import tqdm
import cv2
import numpy as np
import os

# move to the directory with the dataset from the authors
os.chdir('CMFDdb_grip/images')

for degradation in ['noise','scale','rotation','jpeg']:
    if not os.path.exists(degradation):
        os.mkdir(degradation)

In [ ]:
def get_positions(filename):
    with open(filename, 'r') as f:
        for i,line in enumerate(f.readlines()):
            if not i:
                src = line.split(',')
            else:
                target = line.split(',')
                break
            f.close()
    x_src = int(src[0]) - 1
    y_src = int(src[1][:-1]) - 1
    x_target = int(target[0]) - 1
    y_target = int(target[1][:-1]) - 1
    return x_src, y_src, x_target, y_target

def add_gaussian_noise(copy, sigma):
    return np.clip(copy + np.random.normal(0, sigma, copy.shape), 0, 1)

def rotate(copy, angle):
    return np.clip(ndimage.rotate(copy, angle, reshape=True), 0, 1)

def scale(copy, s):
    h, w, _ = copy.shape
    return cv2.resize(copy, (int(s*w),int(s*h)))

def jpeg_compress(copy, qf):
    Image.fromarray((copy[:,:,:3]*255).astype(np.uint8)).save('../jpeg/compressed.jpg', format='JPEG', quality=qf)
    compressed = Image.open('../jpeg/compressed.jpg')
    return np.array(compressed) /255.

files = os.listdir()
noise_values = [.02, .04, .06, .08, .1]
angle_values = [4, 10, 20, 30, 60, 90, 180]
scale_values = [.5, .8, .91, 1.05, 1.09, 1.2, 2]
jpeg_values = [90, 80, 70, 60, 50, 40, 30, 20]

for idx in tqdm(range(80)):
    filenames = files[4*idx:4*(idx+1)]
    # read images
    img = Image.open(filenames[0])
    img = np.array(img) /255.
    copy = Image.open(filenames[2])
    copy = np.array(copy) /255.
    x_src, y_src, x_target, y_target = get_positions(filenames[1])
    forged_name = '../{}/' + filenames[0][:-4] + '_{}_copy' + filenames[0][-4:]
    gt_name = '../{}/' + filenames[0][:-4] + '_{}_gt' + filenames[0][-4:]
    
    H, W, _ = img.shape
    h, w, _ = copy.shape
    gt = np.zeros(img.shape[:2])
    gt[x_src:x_src+h, y_src:y_src+w] += copy[:,:,3]
    
    for sigma in noise_values:
        forged = img.copy()
        gt_forged = gt.copy()
        noisy = add_gaussian_noise(copy, sigma)
        h, w, _ = noisy.shape
        hf, wf = min(H,x_target+h) - x_target, min(W,y_target+w) - y_target
        gt_forged[x_target:x_target+hf, y_target:y_target+wf] = noisy[:hf,:wf,3]
        keep_mask = np.ones((hf,wf)) - noisy[:hf,:wf,3]
        forged[x_target:x_target+hf, y_target:y_target+wf] *= np.expand_dims(keep_mask,2)
        forged[x_target:x_target+hf, y_target:y_target+wf] += noisy[:hf,:wf,:3] * np.expand_dims(noisy[:hf,:wf,3],2)
        Image.fromarray((forged* 255).astype(np.uint8)).save(forged_name.format('noise',str(sigma)))
        Image.fromarray((gt_forged* 255).astype(np.uint8)).save(gt_name.format('noise',str(sigma)))
        
    for angle in angle_values:
        forged = img.copy()
        gt_forged = gt.copy()
        rotated = rotate(copy, angle)
        h, w, _ = rotated.shape
        hf, wf = min(H,x_target+h) - x_target, min(W,y_target+w) - y_target
        gt_forged[x_target:x_target+hf, y_target:y_target+wf] = rotated[:hf,:wf,3]
        keep_mask = np.ones((hf,wf)) - rotated[:hf,:wf,3]
        forged[x_target:x_target+hf, y_target:y_target+wf] *= np.expand_dims(keep_mask,2)
        forged[x_target:x_target+hf, y_target:y_target+wf] += rotated[:hf,:wf,:3] * np.expand_dims(rotated[:hf,:wf,3],2)
        Image.fromarray((forged* 255).astype(np.uint8)).save(forged_name.format('rotate',str(angle)))
        Image.fromarray((gt_forged* 255).astype(np.uint8)).save(gt_name.format('rotate',str(angle)))
        
    for s in scale_values:
        forged = img.copy()
        gt_forged = gt.copy()
        scaled = scale(copy, s)
        h, w, _ = scaled.shape
        hf, wf = min(H,x_target+h) - x_target, min(W,y_target+w) - y_target
        gt_forged[x_target:x_target+hf, y_target:y_target+wf] = scaled[:hf,:wf,3]
        keep_mask = np.ones((hf,wf)) - scaled[:hf,:wf,3]
        forged[x_target:x_target+hf, y_target:y_target+wf] *= np.expand_dims(keep_mask,2)
        forged[x_target:x_target+hf, y_target:y_target+wf] += scaled[:hf,:wf,:3] * np.expand_dims(scaled[:hf,:wf,3],2)
        Image.fromarray((forged* 255).astype(np.uint8)).save(forged_name.format('scale',str(s)))
        Image.fromarray((gt_forged* 255).astype(np.uint8)).save(gt_name.format('scale',str(s)))
        
    for qf in jpeg_values:
        forged = img.copy()
        gt_forged = gt.copy()
        compressed = jpeg_compress(copy, qf)
        h, w, _ = compressed.shape
        hf, wf = min(H,x_target+h) - x_target, min(W,y_target+w) - y_target
        gt_forged[x_target:x_target+hf, y_target:y_target+wf] = copy[:hf,:wf,3]
        keep_mask = np.ones((hf,wf)) - copy[:hf,:wf,3]
        forged[x_target:x_target+hf, y_target:y_target+wf] *= np.expand_dims(keep_mask,2)
        forged[x_target:x_target+hf, y_target:y_target+wf] += compressed[:hf,:wf,:3] * np.expand_dims(copy[:hf,:wf,3],2)
        Image.fromarray((forged* 255).astype(np.uint8)).save(forged_name.format('jpeg',str(qf)))
        Image.fromarray((gt_forged* 255).astype(np.uint8)).save(gt_name.format('jpeg',str(qf)))
os.remove('../jpeg/compressed.jpg')